In [4]:
# Importing Libraries

import pandas as pd
import numpy as np

In [237]:
# Reading Excel data 

df=pd.read_excel(r"C:\Users\Prajwal\Desktop\Symantics\Task01.xlsx")
df.tail()

,Sr no,District,Taluka,Village,Lat,Long,NDVI,Area
358,359,SEONI,KURAI,Sapapar,22.050800,79.530509,0.878235,0.01
359,360,SEONI,KURAI,Sapapar,22.046644,79.530359,0.796163,0.01
360,361,SEONI,KURAI,Sapapar,22.056421,79.522797,0.866143,0.01
361,362,SEONI,KURAI,Sapapar,22.055310,79.519804,0.807326,0.01
362,363,SEONI,KURAI,Sapapar,22.059187,79.529066,0.883400,0.01


# NDVI
NDVI always ranges from -1 to +1. But there isn’t a distinct boundary for each type of land cover.
For example, when you have negative values, it’s highly likely that it’s water. On the other hand, if you have an NDVI value close to +1, there’s a high possibility that it’s dense green leaves.
But when NDVI is close to zero, there are likely no green leaves and it could even be an urbanized area.

**Note: So here we are considering our NDVI range from 0 to +1.**

In [240]:
# Creating Column Crop Status for distinguishing crop on NDVI basis

df['C_Status'] = pd.cut(df.NDVI, bins=[0.0,0.2,0.51,0.71,1.0],
                        labels=['V_POOR','POOR','AVG','GOOD'])
df

,Sr no,District,Taluka,Village,Lat,Long,NDVI,Area,C_Status
0,1,SEONI,Barghat,Bori Khurd,21.934773,79.732698,0.547747,0.01,AVG
1,2,SEONI,Barghat,Bori Khurd,21.919265,79.748639,0.688359,0.01,AVG
2,3,SEONI,Barghat,Bori Khurd,21.918795,79.741822,0.655957,0.01,AVG
3,4,SEONI,Barghat,Bori Khurd,21.930891,79.742183,0.520902,0.01,AVG
4,5,SEONI,Barghat,Bori Khurd,21.918017,79.748603,0.665861,0.01,AVG
...,...,...,...,...,...,...,...,...,...
358,359,SEONI,KURAI,Sapapar,22.050800,79.530509,0.878235,0.01,GOOD
359,360,SEONI,KURAI,Sapapar,22.046644,79.530359,0.796163,0.01,GOOD
360,361,SEONI,KURAI,Sapapar,22.056421,79.522797,0.866143,0.01,GOOD
361,362,SEONI,KURAI,Sapapar,22.055310,79.519804,0.807326,0.01,GOOD


In [7]:
# Count of District
 
t2 = df.pivot_table(index = ['District'], aggfunc ='size')
print(t2)

District
SEONI    363
dtype: int64


In [8]:
# Count of District and Taluka

t1 = df.pivot_table(index = ['District','Taluka'], aggfunc ='size')
print(t1)
print("Count: ",t1.count())

District  Taluka   
SEONI     Barghat       48
          Chhapara      47
          Dhanora       41
          Ghanusaur     44
          KURAI        145
          Keolari       38
dtype: int64
Count:  6


In [12]:
# Count of Villages

t3 = df.pivot_table(index = ['Village'], aggfunc ='size')
print(t3)
print("\nVillage Count: ",t3.count())

Village
Agariya Kalan         21
Amajhiri              11
Anjaniya              36
Badalpar              22
Bamhanwada             6
Bori Khurd            25
Bothiya (Pkh 039)     38
Ghateri               22
Kudobudhawara         23
Pouniya               23
Ramkhairi             13
Sapapar              112
Sukri                 11
dtype: int64

Village Count:  13


In [241]:
# Creating Column for  Crop Status Count  
df['Status_Count']=df.groupby(['Village','C_Status'])['C_Status'].transform('count')

# Creating Column for total farmers in a particular village
df['V_Count']=df.groupby(['Village'])['C_Status'].transform('count')

# Creating Percentage column for Village Crop Status
df['Percent']=df['Status_Count']/df['V_Count']*100

a = df.pivot_table(index = ['Village','C_Status'], aggfunc ='size')
a

Village            C_Status
Agariya Kalan      V_POOR        0
                   POOR          2
                   AVG          12
                   GOOD          7
Amajhiri           V_POOR        0
                   POOR          0
                   AVG           1
                   GOOD         10
Anjaniya           V_POOR        0
                   POOR          1
                   AVG           9
                   GOOD         26
Badalpar           V_POOR        0
                   POOR          0
                   AVG           6
                   GOOD         16
Bamhanwada         V_POOR        0
                   POOR          0
                   AVG           1
                   GOOD          5
Bori Khurd         V_POOR        0
                   POOR          5
                   AVG          14
                   GOOD          6
Bothiya (Pkh 039)  V_POOR        0
                   POOR          0
                   AVG           5
                   GOOD    

In [242]:
# Fuction For Proportional Sampling of farmers based on each village according to Crop Status weightage(percent).

def f(p):    
    
    df['Possibility']=round(df['Percent']*p/100)
    df['Sample']=p
    df['Selected Farmers'] = df['Sample'].apply(np.ceil).astype(int)
    df['Selected Farmers'] = df.groupby('Village').apply(lambda df: df['C_Status'].sample(df['Selected Farmers'].iat[0])).reset_index(level=0)['Village']
    df['Selected Farmers'] = df['Selected Farmers'].fillna('')
    print(df)
      
f(5)

     Sr no District   Taluka     Village        Lat       Long      NDVI  \
0        1    SEONI  Barghat  Bori Khurd  21.934773  79.732698  0.547747   
1        2    SEONI  Barghat  Bori Khurd  21.919265  79.748639  0.688359   
2        3    SEONI  Barghat  Bori Khurd  21.918795  79.741822  0.655957   
3        4    SEONI  Barghat  Bori Khurd  21.930891  79.742183  0.520902   
4        5    SEONI  Barghat  Bori Khurd  21.918017  79.748603  0.665861   
..     ...      ...      ...         ...        ...        ...       ...   
358    359    SEONI    KURAI     Sapapar  22.050800  79.530509  0.878235   
359    360    SEONI    KURAI     Sapapar  22.046644  79.530359  0.796163   
360    361    SEONI    KURAI     Sapapar  22.056421  79.522797  0.866143   
361    362    SEONI    KURAI     Sapapar  22.055310  79.519804  0.807326   
362    363    SEONI    KURAI     Sapapar  22.059187  79.529066  0.883400   

     Area C_Status  Status_Count  V_Count    Percent  Possibility  Sample  \
0    0.01 

In [ ]:
# Creating and Saving Sample Data with all farmers data

df.to_excel(r'C:\Users\Prajwal\Desktop\Symantics\task2.xlsx')
print("Saved !!!")

In [244]:
# Fuction directectly provides required data neglecting all i.e. only samples from all villages.

def f1(p):    
    
    df['Possibility']=round(df['Percent']*p/100)
    df['Sample']=p
    df['Selected Farmers'] = df['Sample'].apply(np.ceil).astype(int)
    df['Selected Farmers'] = df.groupby('Village').apply(lambda df: df['C_Status'].sample(df['Selected Farmers'].iat[0])).reset_index(level=0)['Village']
    df['Selected Farmers'] = df['Selected Farmers'].fillna('')
    df['Selected Farmers'].replace('', np.nan, inplace=True) 
    df.dropna(subset=['Selected Farmers'], inplace=True)
    print(df)
    
        
f1(5)


     Sr no District   Taluka     Village        Lat       Long      NDVI  \
9       10    SEONI  Barghat  Bori Khurd  21.897307  79.729557  0.281047   
11      12    SEONI  Barghat  Bori Khurd  22.000452  79.772478  0.799268   
12      13    SEONI  Barghat  Bori Khurd  22.009416  79.772571  0.852725   
15      16    SEONI  Barghat  Bori Khurd  21.989044  79.911217  0.331414   
21      22    SEONI  Barghat  Bori Khurd  21.955359  79.788631  0.605782   
..     ...      ...      ...         ...        ...        ...       ...   
286    287    SEONI    KURAI     Sapapar  22.654489  79.631012  0.820367   
297    298    SEONI    KURAI     Sapapar  22.582273  79.793769  0.851044   
329    330    SEONI    KURAI     Sapapar  22.285561  79.735863  0.722449   
338    339    SEONI    KURAI     Sapapar  22.315266  79.722781  0.874082   
358    359    SEONI    KURAI     Sapapar  22.050800  79.530509  0.878235   

     Area C_Status  Status_Count  V_Count    Percent  Possibility  Sample  \
9    0.01 

In [225]:
# Creating and Saving only Sample Data

df.to_excel(r'C:\Users\Prajwal\Desktop\Symantics\task3.xlsx')
print("Saved !!!")

Saved !!!


In [233]:
# Rechecking Final Count of DataFrame.

df.pivot_table(index = ['Village','Selected Farmers'], aggfunc ='size')
df.count()


Sr no               65
District            65
Taluka              65
Village             65
Lat                 65
Long                65
NDVI                65
Area                65
C_Status            65
Status_Count        65
V_Count             65
Possibility         65
Percent             65
Selected Farmers    65
Sample              65
dtype: int64